# Packages

In [ ]:
import pandas as pd
from functions_alt import news_data, full_url

# 1. Altmetric mentions

In [ ]:
df_am = pd.read_csv('data/altmetric_mentions.csv', sep=',', encoding='UTF-8')

There are 1513 news mentions.

In [ ]:
df_am = df_am[df_am['Mention Type']=='News story'].reset_index(drop=True)
df_am.shape

But, 139 do not include the news media URL, so they are removed.

In [ ]:
df_am = df_am[~df_am['Mention URL'].isna()].reset_index(drop=True)
df_am.shape

# 2. Site Traffic API

In [ ]:
df_am['Clean URL'] = df_am['Mention URL']

In [ ]:
df_am['Clean URL'] = df_am['Clean URL'].str.replace('(http://)|(https://)|(www.)', '', regex=True)
df_am['Clean URL'] = df_am['Clean URL'].str.replace('/.*', '', regex=True)

There are 447 news media.

In [ ]:
len(list(set(df_am['Clean URL'].tolist())))

## 2.1. API query 1

All URLs.

In [ ]:
url = 'https://zylalabs.com/api/29/site+traffic+api/93/traffic+source+and+overview'
key = ''
header = {'Authorization':key}

In [ ]:
df_news = news_data(url, header, list(set(df_am['Clean URL'].tolist())))

In [ ]:
df_news.to_csv('data/News/media_metrics.tsv', sep='\t', encoding='UTF-8', index=False)

## 2.2. API query 2

Short URLs are fixed.

In [ ]:
df_am['Full URL'] = None
df_am.loc[df_am['Clean URL']=='ct.moreover.com', 'Full URL'] = df_am.loc[df_am['Clean URL']=='ct.moreover.com', 'Mention URL']

In [ ]:
for i in df_am[df_am['Clean URL']=='ct.moreover.com'].index:
    df_am.loc[i, 'Full URL'] = full_url(df_am.loc[i, 'Mention URL'])

In [ ]:
df_am['Full URL'] = df_am['Full URL'].str.replace('(http://)|(https://)|(www.)', '', regex=True)
df_am['Full URL'] = df_am['Full URL'].str.replace('/.*', '', regex=True)

In [ ]:
df_am.loc[df_am['Clean URL']=='ct.moreover.com', 'Full URL']

In [ ]:
len(list(set(df_am['Full URL'].tolist())))

In [ ]:
len(set(df_am[df_am['Full URL'].isin(df_am['Clean URL'])]['Full URL']))

In [ ]:
len(set(df_am[(~df_am['Full URL'].isin(df_am['Clean URL'])) & (~df_am['Full URL'].isna())]['Full URL']))

In [ ]:
url = 'https://zylalabs.com/api/29/site+traffic+api/93/traffic+source+and+overview'
key = ''
header = {'Authorization':key}

In [ ]:
df_news_full = news_data(url, header, list(set(df_am[(~df_am['Full URL'].isin(df_am['Clean URL'])) & (~df_am['Full URL'].isna())]['Full URL'])))

In [ ]:
df_news_full.to_csv('data/News/media_metrics_full.tsv', sep='\t', encoding='UTF-8', index=False)

In [ ]:
df_am.loc[df_am['Clean URL']=='ct.moreover.com', 'Clean URL'] = df_am.loc[df_am['Clean URL']=='ct.moreover.com', 'Full URL']

In [ ]:
df_am.to_csv('data/News/news_mentions_full.tsv', sep='\t', index=False, encoding='UTF-8')